In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from itertools import product
import multiprocessing as mp
mp.set_start_method('spawn',True)
torch.multiprocessing.set_start_method('spawn',True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Machine_Replacement:
    def __init__(self,rep_cost=0.7,nS=6,nA=2):
        self.nS = nS;
        self.nA = nA;
        self.cost = np.linspace(0.1, 0.99,nS);
        self.rep_cost = rep_cost;
    def gen_probability(self):
        self.P = np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            for j in range(self.nS):
                if(i<=j):
                    self.P[0,i,j]=(i+1)*(j+1);
                else:
                    continue;
            self.P[0,i,:]=self.P[0,i,:]/np.sum(self.P[0,i,:])
            self.P[1,i,0]=1;
        return self.P;
    def gen_reward(self):
        self.R=np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            self.R[0,i,:] = self.cost[i];
            self.R[1,i,0] = self.rep_cost+self.cost[0];
        return self.R;
    def gen_expected_reward(self):
        self.R = np.zeros((self.nA,self.nS));
        for i in range(self.nS):
            self.R[0,i] = self.cost[i];
            self.R[1,i] = self.rep_cost + self.cost[0];
        return self.R;

In [21]:
class Target_Policy:
    '''
        First we create an initiualizer function namely a constructor to initialize the variables
        with initial data values
    '''
    def __init__(self,S,A,P,R,start):
        self.S=S # represant the states of the MDP
        self.nS = len(S) # Reperesants the number of states of the MDP
        self.nA = len(A);# Represants the number of actions in the MDP
        self.P=P # Represants the true Probability function
        self.R=R # Represants the true Reward  function
        self.A=A;# Represnats the Action Space
        self.K_pol = []; # To store all the policies
        self.s_start=start # Store the start state 
    '''
        In the generate_next_state(), we are generating our next state to be visited based on the following input parameters
        s : Current state
        a : Current action
    '''    
    def generate_next_state(self,s,a):
        #p = np.zeros(self.nS)
        p = self.P[a][s][:] # extrcat all the probabilities of the different statestransition given current s and a
        #print(p);
        return (np.argmax(np.random.multinomial(1,p)))
    
    '''
        Single function to find the plot between the cumulative regret generated by different algorithms
        Parameters:
            reg_list : A list containing the regret value at different runs instances averaged over several time
    '''    
    def plot_data(self,reg_list):
        plt.plot(np.arange(len(reg_list)),np.cumsum(np.array(reg_list)),marker = '+'); 
    '''
        Function to find the optimum policy out of the K policies found out.
        Parameters:
            runs : To find for how many runs the current policy to be runned
            T : Each run consisiting of how many time steps to find the average reward for each policy in one run
            Time complexity : O(#(policies) x #(episode runs) x #(number of Time steps in one episode))
    '''
    def find_optimum_policy(self):
        self.find_policies(); #Call the find_policies() to find all the policies and store it in 'self.K' list
        final_R = np.zeros(len(self.K_pol));
        for idx,pol in enumerate(self.K_pol):
            #policy = self.one_hot(pol);
            beh_obj = beh_pol_sd(self.P, pol, self.nS, self.nA)
            state_distribution = beh_obj.state_distribution_simulated(1);
            final_R[idx] = sum([state_distribution[state] *self.R[int(pol[state]),state] for state in range(self.nS)]);
        for l_pol in range(len(self.K_pol)):
            print(self.K_pol[l_pol],"    ====>    ",final_R[l_pol]); # Display the the expected reward for each policy
        return (final_R,self.K_pol[np.argmin(final_R)],np.min(final_R));# Return the minimum reward, the policy number which gives the minimum reward and the policy that gives minimum reward
    
    def find_policies(self):
        self.K_pol = [];
        pol=np.zeros(self.nS) # First policy is all 0's
        self.K_pol.append(np.array(pol)); # append it to our K_policy list namely self.K
        for i in reversed(range(self.nS)):
            pol[i] = 1; # Come from the end and since the structure is thresholding nature so make each position 1 from 0 and append the same
            print(pol);
            self.K_pol.append(np.array(pol));
        print(len(self.K_pol)," policies found");

In [22]:
class get_hyperparameters:
    def __init__(self):
        self.T = 500000;
        self.runs = 10;
        self.lr = 0.1;
        self.batch_size = 50;
        self.start = 0;
        self.nS = 4;
        self.nA = 2;
        self.rep_cost = 0.7
        self.alpha = 0.2
        self.gamma = 0.95
    
    def ret_hyperparameters(self):
        return (self.T,self.runs,self.lr,self.batch_size,self.start,self.nS,self.nA,self.rep_cost,self.alpha,self.gamma)

In [23]:
class beh_pol_sd:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,onehot_encode=1):
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
'''
    The class weights is created to define the neural network structure.
    Inputs:
    -------
    input_size  : number of input head perceptrons. Basically is equal to number of states nS.
    output_size : number of perceptrons in the output layer.
'''
class weights(nn.Module):
    def __init__(self,input_size,output_size,hidden_size = 0):
        super(weights,self).__init__()
        self.input_size = input_size;
        self.hidden_size = hidden_size;
        self.output_size = output_size;
        if(hidden_size!=0):
            self.linear1 = nn.Linear(self.input_size, self.hidden_size, bias=False)
            self.linear2 = nn.Linear(self.hidden_size, self.output_size, bias=False)
        else:
            self.linear1 = nn.Linear(self.input_size, self.output_size, bias=False)
    '''
        forward(): We accept a state 's' as input. Then we convert this into one hot encoding which is accomplished by first two lines.
        Further we convert this one_hot vector 's' into pytorch tensor and then pass it through the network to obtain a output which is returned 
    '''
    def forward(self,state):
        s = np.zeros(self.input_size);
        #print(state,end='===>');
        s[state] = 1;
        state = torch.FloatTensor(s).to(device)
        #print(state);
        if(self.hidden_size == 0):
            output = torch.exp(self.linear1(state)) #To ensure that the outputs are always positive. giving Relu will cause problems.
        else:
            output = torch.exp(self.linear2(torch.exp(self.linear1(state))));
        return output

In [25]:
class average_case_distribution:
    def __init__(self,nS,nA,behaviour_policy,state,lr):
        self.nS = nS
        self.nA = nA
        self.behaviour_policy = behaviour_policy;
        self.state = state;
        self.lr = lr
        self.W_loss = 0
        self.weight_obj = weights(nS,1).to(device);
        self.W_loss = 0;
    def set_target_policy(self,target_pol):
        self.target_policy = target_pol;
        self.optimizerW = optim.Adam(self.weight_obj.parameters(),lr = self.lr);
        self.batch_size = 50
    def show_policy(self):
        print(self.target_policy);
    def set_batch(self,data):
        self.data = data;
        self.T = len(data);
    def get_batch(self):
        if(self.T<=50):
            return self.data
        else:
            i = 1;
            j = np.random.choice(self.T);
            batch = [];
            while(i<=self.batch_size):
                if(np.random.random()<=0.5):
                    batch.append([self.data[j][0],self.data[j][1],self.data[j][2]])
                    j = (j+1)%self.T;
                    i+=1;
            return batch;
    
    def get_w(self,data,weight_obj,m,pair=0):
        if(pair == 1):
            Z_w_state = 0;
            for i in range(len(data)):
                val = weight_obj(data[i][0]);
                #print(val);
                Z_w_state+=val;
            #print(Z_w_state.detach().numpy()[0]/self.batch_size);
            return Z_w_state.cpu().detach().numpy()[0]/self.batch_size;
        else:
            state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2 = list(),list(),list(),list(),list(),list(),list(),list();
            K = list();
            for i in range(len(data)):
                sample1 = data[i][0];
                sample2 = data[i][1];
                state1.append(sample1[0]);
                #print(sample1);
                w_state1.append(weight_obj(sample1[0]));
                w_next_state1.append(weight_obj(sample1[2]));
                state2.append(sample2[0]);
                w_state2.append(weight_obj(sample2[0]));
                w_next_state2.append(weight_obj(sample2[2]));
                beta1.append(self.target_policy[sample1[0],sample1[1]]/self.behaviour_policy[sample1[0],sample1[1]]);
                beta2.append(self.target_policy[sample2[0],sample2[1]]/self.behaviour_policy[sample2[0],sample2[1]]);
                K.append(sample1[2]==sample2[2]);
            return (state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K);
    
    def get_state_distribution_ratio(self):
        batch = self.get_batch();
        pairs = list(product(batch,repeat=2));
        state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K = self.get_w(pairs, self.weight_obj, len(batch));
        Z_w_state = self.get_w(batch, self.weight_obj, len(batch),1);
        self.w_loss = 0
        for i in range(len(state1)):
            self.w_loss+=(beta1[i]*(w_state1[i]/Z_w_state) - (w_next_state1[i]/Z_w_state))*(beta2[i]*(w_state2[i]/Z_w_state)-(w_next_state2[i]/Z_w_state))*K[i];
        self.w_loss/=(2*self.batch_size);
        self.optimizerW.zero_grad();
        self.w_loss.backward();
        self.optimizerW.step();
        self.optimizerW.zero_grad();
        state_dist=[];
        for i in range(self.nS):
            w_state = self.weight_obj(i);
            w_state = w_state.cpu().detach().numpy()[0];
            state_dist.append(w_state);
        return np.array(state_dist);

In [26]:
def one_hot(target_policy,nS,nA):
    one_hot_tp = [];
    for i in range(len(target_policy)):
        policy = target_policy[i];
        print(policy);
        tp=np.zeros((nS,nA));
        for j in range(nS):
            tp[j][policy[j]] = 1;
        one_hot_tp.append(tp);
    return np.array(one_hot_tp);

In [30]:
def processing(T,run,T_update,batch_size,nS,nA,behaviour_policy,target_policy,one_hot_target_policy):
    nPOL = nS
    lr =0.1
    ac_obj = list();
    value_function=np.zeros(T_update)
    #start = 0;
    for i,pol in enumerate(one_hot_target_policy):
        ac_obj.append( average_case_distribution(nS, nA, behaviour_policy, 0, lr))
        ac_obj[i].set_target_policy(pol);
    data = list();
    policy_chose=np.zeros(nS)
    state = 0;
    #state_distribution = np.array([[np.random.random() for _ in range(nS)] for i in range(nPOL)])
    print("Running")
    for t in range(1,T+batch_size+1):
        if(t%batch_size==0):
            #c+=1;
            #print(k);
            min_pol=np.zeros(nPOL);
            for i in range(nPOL):
                ac_obj[i].set_batch(np.array(data));
                sd = ac_obj[i].get_state_distribution_ratio();
                sd = sd*behaviour_state_distribution;
                sd = sd/np.sum(sd);
                #sd = state_distribution[i];
                #print(i,"==>",sd);
                min_pol[i]=sum([R[target_policy[i,s],s]*sd[s] for s in range(nS)]);
            k = np.argmin(min_pol);
            #print(min_pol,k);
            policy_chose[k]+=1;
            vf = true_value_function_list[k+1];
            #data_dict[run]=data_dict[run]+data;
            #data = [];
            value_function[int(t/batch_size)-1] = vf;
            #print(k," ==> ",vf);
            #input();
        action = np.argmax(np.random.multinomial(1,behaviour_policy[state,:]));
        next_state = np.argmax(np.random.multinomial(1,P[action,state,:]));
        data.append([state,action,next_state]);
        state = next_state;
    with open("Machine_rep_"+str(nS)+"_states_data_run"+str(run),'wb') as f:
        pickle.dump(data,f);
    f.close();
    with open("Machine_rep_"+str(nS)+"_states_value_func"+str(run),'wb') as f:
        pickle.dump(value_function,f)
    f.close()

In [37]:
if __name__=='__main__':
    T,runs,lr,batch_size,start,nS,nA,rep_cost,alpha,gamma = get_hyperparameters().ret_hyperparameters();
    print(get_hyperparameters().ret_hyperparameters())
    nPOL = nS;
    eps = 0.08;
    epsilon = 0.1;
    mr_obj = Machine_Replacement(rep_cost,nS,nA);
    P,R,R_comp = mr_obj.gen_probability(),mr_obj.gen_expected_reward(),mr_obj.gen_reward();
    behaviour_policy = np.ones((nPOL,nA))*0.5
    behaviour_state_distribution = beh_pol_sd(P, behaviour_policy, nS, nA).state_distribution_simulated(0)
    target_policy = np.ones((nPOL,nS),dtype = np.int8)
    for i in range(nPOL):
        target_policy[i][:-(i+1)] = 0;

    epsilon_optimal_cost_matrix = np.zeros((int(T/batch_size),runs));
    value_function = np.zeros((int(T/batch_size),runs));
    #q_learning_optimal_cost_matrix = np.zeros((T,runs));
    tp=Target_Policy(np.arange(nS), np.arange(nA), P, R, start)
    true_value_function_list,optimal_policy,optimal_value_function = tp.find_optimum_policy();
    one_hot_target_policy = one_hot(target_policy,nS,nA);
    policy_chose=np.zeros(nS)
    ac_obj = list();
    T_update = int(T/batch_size);
    for run in range(runs):
        processing(T,run,T_update,batch_size,nS,nA,behaviour_policy,target_policy,one_hot_target_policy);
    

(500000, 10, 0.1, 50, 0, 4, 2, 0.7, 0.2, 0.95)
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 1. 1. 1.]
[1. 1. 1. 1.]
5  policies found
[0. 0. 0. 0.]     ====>     0.9900000000000003
[0. 0. 0. 1.]     ====>     0.4907944514501892
[0. 0. 1. 1.]     ====>     0.42741721854304665
[0. 1. 1. 1.]     ====>     0.4315789473684213
[1. 1. 1. 1.]     ====>     0.7999999999999999
[0 0 0 1]
[0 0 1 1]
[0 1 1 1]
[1 1 1 1]
Running


KeyboardInterrupt: 